In [1]:
import numpy as np
import arepo
import sys
from tqdm import tqdm
import astropy.units as u
import h5py as h5
import glob
import os
from numba import njit
import re
from sklearn.cluster import KMeans

from joblib import Parallel, delayed


In [2]:
def read_snap(path, idx, parttype=[0], fields=['Coordinates', 'Masses', 'Velocities', 'ParticleIDs']):
    
    fname = path + '/output'
    
    return arepo.Snapshot(fname, idx, parttype=parttype, fields=fields, combineFiles=True)

In [12]:
path = '../runs/Nbody/lvl4/'
nsnap = 1600
nchunk = 64

indices = np.arange(nsnap)
    
sn = read_snap(path, indices[-1], parttype=[2])
ids = sn.part2.id
ids = np.sort(ids)

# open a file and load dummy data for each chunk of stars
id_chunks = np.array_split(ids, nchunk)

In [26]:
h5dummy = h5.File('/n/holystore01/LABS/hernquist_lab/Users/abeane/starbar_runs/phase_space_data/tmp.h5', mode='w')
id_chunk_list = id_chunks[0]
pos = np.zeros((len(id_chunk_list), nsnap, 3))
vel = np.zeros((len(id_chunk_list), nsnap, 3))

h5dummy.create_dataset('Coordinates', data=pos)
h5dummy.create_dataset('Velocities', data=vel)

<HDF5 dataset "Velocities": shape (12587, 1600, 3), type "<f8">

CPU times: user 170 ms, sys: 221 ms, total: 392 ms
Wall time: 1.42 s


In [3]:
prefix = '/n/home01/abeane/starbar/plots/phase_space/data_tmp/Nbody-lvl4/'
t = h5.File(prefix+'phase_space_Nbody-lvl4.0.hdf5', mode='r')

In [4]:
print(t['Coordinates'][0][:10])
print(t['ParticleIDs'][0])

[[ 1.93207335  0.38477442 -0.00709001]
 [ 1.90658453  1.23832436  0.26236722]
 [ 1.52180065  1.78986435  0.31239718]
 [ 0.88375337  2.03913904  0.13625232]
 [ 0.08057285  1.90360447 -0.1523453 ]
 [-0.79874971  1.2518507  -0.28543778]
 [-1.32684023  0.25484387 -0.08740754]
 [-1.36765257 -0.83543199  0.23344814]
 [-1.00780701 -1.69711542  0.35551181]
 [-0.37503367 -2.23172422  0.18917092]]
3168481


In [20]:
center = np.array([0., 0., 0.])
pos = np.array(t['Coordinates']) - center
vel = np.array(t['Velocities'])
w = np.concatenate((pos, vel), axis=-1)
w = np.swapaxes(w, 0, 1)

In [21]:
print(np.shape(w))

(1601, 12587, 6)


In [15]:
print(np.shape(pos))

(12587, 1601, 3)


In [22]:
idx = 3
sn = read_snap(path, idx, parttype=[2])

key = int(np.where(sn.part2.id == t['ParticleIDs'][0])[0])
sn.part2.pos[key]

Quantity([0.88375337, 2.03913904, 0.13625232], unit=[kpc])